<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/complete_singularity_ai_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
"""
complete_singularity_ai.py

Self-contained script that:
1. Generates a synthetic SingularityDynamicsDataset
2. Defines an EnhancedSingularityAI with LayerNorm and Dropout
3. Trains with MSE loss on (inputs @ A + nonlin(inputs) @ B + noise)
4. Performs MC-Dropout inference for uncertainty
"""

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import numpy as np

# 1. Synthetic Dataset
class SingularityDynamicsDataset(Dataset):
    def __init__(self, num_samples=5000, input_dim=6, output_dim=3):
        super().__init__()
        self.inputs = torch.randn(num_samples, input_dim)
        # Linear mapping A: (input_dim → output_dim)
        A = torch.randn(input_dim, output_dim)
        # Nonlinear mapping B for sin term
        B = torch.randn(input_dim, output_dim)
        # Compute outputs: linear + sin-nonlinear + noise
        lin_part = self.inputs @ A                            # [N, output_dim]
        nonlin   = torch.sin(self.inputs) @ B                 # [N, output_dim]
        noise    = 0.05 * torch.randn(num_samples, output_dim)
        self.outputs = lin_part + 0.1 * nonlin + noise        # [N, output_dim]

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.outputs[idx]

# 2. Enhanced Model with LayerNorm & Dropout
class EnhancedSingularityAI(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout_p=0.2):
        super().__init__()
        self.fc1   = nn.Linear(input_dim,  hidden_dim)
        self.ln1   = nn.LayerNorm(hidden_dim)
        self.act1  = nn.SELU()
        self.drop1 = nn.Dropout(dropout_p)

        self.fc2   = nn.Linear(hidden_dim, hidden_dim)
        self.ln2   = nn.LayerNorm(hidden_dim)
        self.act2  = nn.SELU()
        self.drop2 = nn.Dropout(dropout_p)

        self.fc3   = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.drop1(self.act1(self.ln1(self.fc1(x))))
        x = self.drop2(self.act2(self.ln2(self.fc2(x))))
        return self.fc3(x)

# 3. Training Routine
def train_model(model, loader, epochs=50, lr=1e-3, device='cpu'):
    model.to(device)
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    for epoch in range(1, epochs+1):
        model.train()
        total_loss = 0.0
        for xb, yb in loader:
            xb, yb = xb.to(device), yb.to(device)
            opt.zero_grad()
            pred = model(xb)
            loss = criterion(pred, yb)
            loss.backward()
            opt.step()
            total_loss += loss.item() * xb.size(0)
        avg = total_loss / len(loader.dataset)
        if epoch == 1 or epoch % 10 == 0:
            print(f"Epoch {epoch:3d} | Loss {avg:.6f}")

# 4. MC-Dropout Inference
def mc_dropout_inference(model, x, mc_iters=100):
    model.train()  # keep dropout on
    preds = []
    with torch.no_grad():
        for _ in range(mc_iters):
            preds.append(model(x).cpu().numpy())
    arr = np.stack(preds, axis=0)      # [mc_iters, batch, output_dim]
    return arr.mean(0), arr.std(0)

# 5. Main Execution
def main():
    # Hyperparameters
    input_dim, hidden_dim, output_dim = 6, 64, 3
    batch_size, epochs, lr = 128, 100, 1e-3

    # Data
    ds = SingularityDynamicsDataset(
        num_samples=5000,
        input_dim=input_dim,
        output_dim=output_dim
    )
    loader = DataLoader(ds, batch_size=batch_size, shuffle=True)

    # Model
    model = EnhancedSingularityAI(
        input_dim, hidden_dim, output_dim, dropout_p=0.2
    )

    # Train
    train_model(model, loader, epochs=epochs, lr=lr, device='cpu')

    # Inference
    sample = torch.randn(10, input_dim)
    mean_pred, std_pred = mc_dropout_inference(model, sample, mc_iters=100)

    print("Mean predictions:\n", mean_pred)
    print("Prediction std deviations:\n", std_pred)

if __name__ == "__main__":
    main()